# Importing required liberaries

In [1]:
# importing required liberaries
import requests
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img
from sklearn.externals import joblib
from IPython.display import clear_output
from collections import Counter
import tensorflow as tf

import efficientnet.tfkeras as efn
from skimage.color import rgb2lab, lab2rgb
from skimage.transform import resize
from skimage.io import imsave
import numpy as np
import tensorflow as tf
from sklearn.externals import joblib
from skimage.color import rgb2gray
# importing lberaries for model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,Dense,MaxPooling2D,BatchNormalization,GlobalAveragePooling2D,Dropout,Flatten,GaussianNoise
from tensorflow.keras.optimizers import Adam
import seaborn as sns
from sklearn.metrics import confusion_matrix

C:\Users\91946\anaconda3\envs\gputest\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


Loading our pretrained model 

In [2]:
with open('saver/mobilenetv2_complete_dataset.json','r') as json_file:

    json_SavedModel = json_file.read()
model = tf.keras.models.model_from_json(json_SavedModel)
model.load_weights('saver/mobilenetv2_complete_dataset.h5')


In [3]:
import gc
gc.collect()

37

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_96 (Model)  (None, 3, 3, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
flatten (Flatten)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               163968    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 387       
Total params: 2,422,339
Trainable params: 2,388,227
Non-trainable params: 34,112
_________________________________________

Decoding the Gestures and converting them into numbers

In [5]:
import copy
def decoding_gestures(class_p1,class_p2,score_p1,score_p2):
    class_p1_copy = copy.copy(class_p1[:])
    class_p1_copy = np.array(class_p1_copy).reshape(-1,3)
    p1 = np.bincount(np.argmax(class_p1_copy,axis =1)).argmax()

    class_p2_copy = copy.copy(class_p2[:])
    class_p2_copy = np.array(class_p2_copy).reshape(-1,3)
    p2 = np.bincount(np.argmax(class_p2_copy,axis =1)).argmax()
    txt,score_p1,score_p2 = declearation(p1,p2,score_p1,score_p2)
    
    return txt,score_p1,score_p2

Setting up the game rules

In [6]:
def declearation(p1,p2,score_p1,score_p2):    
    if p1==0 and p2 == 1:
        txt = 'p1: Is decleared Winner'
        score_p1 = score_p1 + 1

    elif p1==1 and p2 == 0:
        txt = 'p2: Is decleared Winner'
        score_p2 = score_p2 + 1

    elif p1==0 and p2 == 2:
        txt = 'p2: Is decleared Winner'
        score_p2 = score_p2 + 1

    elif p1==2 and p2 == 0:
        txt = 'p1: Is decleared Winner'    
        score_p1 = score_p1 + 1

    elif p1==1 and p2 == 2:
        txt = 'p1: Is decleared Winner'
        score_p1 = score_p1 + 1

    elif p1==2 and p2 == 1:
        txt = 'p2: Is decleared Winner' 
        score_p2 = score_p2 + 1

    else :
        txt = "Its a draw"
        
    return txt,score_p1,score_p2

In [8]:
# setting up the values to have track on the game  
class_p1 = []
class_p2 = []
i = 0
score_p1 = 0
score_p2 = 0

# View the Game 

In [49]:

# we use the ip webcam for using mobile camera for displaing on our pc 
while True:
    
    # getting the address for our mobile camera
    
    # here instead of 19x.xxx.xx.x:8080 you write your own ip given by ip webcam for your android
    image = requests.get("http://19x.xxx.xx.x:8080/shot.jpg") 
    
    # conveerting into numpy array
    video = np.array(bytearray(image.content),dtype = np.uint8)
    
    # decoding our numpy array
    render = cv2.imdecode(video,-1)
    #print(render.shape)
    
    # Reading an image in default mode 
   
    # Window name in which image is displayed 
    window_name = 'Image'
    # Start coordinate, here (5, 5) 
    # represents the top left corner of rectangle 
    start_point_p1 = (0, 0) 
    start_point_p2 = (render.shape[1]//2, 0)
    
    # Ending coordinate, here (220, 220) 
    # represents the bottom right corner of rectangle 
    end_point_p1 = (render.shape[1]//2, render.shape[0]) 
    end_point_p2 = (render.shape[1], render.shape[0])
    
    # Blue color in BGR 
    color = (255, 0, 0) 
    
    # Line thickness of 2 px 
    thickness = 2
    
    # ==========================================================================
    
    # getting the gesture showed by player 1
    var_p1 = render[start_point_p1[1]:end_point_p1[1],start_point_p1[0]:end_point_p1[0],:]
    
    # converting the image from BGR to RGB
    var_p1 = cv2.cvtColor(var_p1, cv2.COLOR_BGR2RGB)
    
    # converting image to array
    var_p1 = array_to_img(var_p1)
    
    # resizing the image
    var_p1 = var_p1.resize((96,96))
    var_p1 = np.array(img_to_array(var_p1))
    
    # normalizing the image 
    var_p1 = var_p1/255
    
    # to avoid change in rows and columns series we used a list to increase one dimension 
    test1 = []
    test1.append(var_p1)
    x_p1 = np.array(test1)
    
    # predicting the output 
    pred_p1  = model.predict(x_p1)
    class_p1.append(pred_p1)
     
    
    # Using cv2.rectangle() method 
    # Draw a rectangle with blue line borders of thickness of 2 px 
    image = cv2.rectangle(render, (0,0), (render.shape[1]//2 , render.shape[1]), color, thickness)
    
    # text to be displayed according to the class value
    if np.argmax(pred_p1)==0:
        text_p1 ="paper"
    if np.argmax(pred_p1)==1:
        text_p1 ="stone"
    if np.argmax(pred_p1)==2:
        text_p1 ="sc"
        
    # putting the text to display the player 1    
    cv2.putText(img = render, 
                text = "p1 : " + text_p1, 
                org = (int(200/2 - 20),int(200/2)), 
                fontFace = cv2.FONT_HERSHEY_SIMPLEX, 
                fontScale = 2, 
                thickness = 2,
                color = (0, 100, 255))

    #=====================================================================================

    # The same process will happen with the player p2
    
    var_p2 = render[start_point_p2[1]:end_point_p2[1],start_point_p2[0]:end_point_p2[0],:]
    
    var_p2 = cv2.cvtColor(var_p2, cv2.COLOR_BGR2RGB)
    var_p2 = array_to_img(var_p2)
    
    var_p2 = var_p2.resize((96,96))
    var_p2 = np.array(img_to_array(var_p2))
    
    
    var_p2 = var_p2/255
    test2 = []
    test2.append(var_p2)
    x_p2 = np.array(test2)
    
    
    pred_p2  = model.predict(x_p2)
    class_p2.append(pred_p2) 
    
    
    if np.argmax(pred_p2)==0:
        text_p2 ="paper"
    if np.argmax(pred_p2)==1:
        text_p2 ="stone"
    if np.argmax(pred_p2)==2:
        text_p2 ="sc"
        
    # putting text to display the player p2
    cv2.putText(img = render, 
                text = "p2 : " + text_p2, 
                org = (int(1200/2 - 20),int(200/2)), 
                fontFace = cv2.FONT_HERSHEY_SIMPLEX, 
                fontScale = 2,                 
                thickness = 2, 
                color = (189, 255, 0))
    # displaying the video frame with detected faces

    i = i+1

    if i%10 ==0:
        txt,score_p1,score_p2 = decoding_gestures(class_p1,class_p2,score_p1,score_p2)
        print(txt)
        
        # putting text to display the declear the winning player
        cv2.putText(img = render, 
                    text = txt, 
                    org = (int(600/2 - 20),int(1200/2)), 
                    fontFace = cv2.FONT_HERSHEY_SIMPLEX, 
                    fontScale = 2, 
                    thickness = 2,
                    color = (0, 0, 255))
        class_p1 = []
        class_p2 = []
    
    # text to display the score of player 1
    cv2.putText(img = render, 
                text = "score : " + str(score_p1), 
                org = (int(200/2 - 20),int(300/2)), 
                fontFace = cv2.FONT_HERSHEY_SIMPLEX, 
                fontScale = 1, 
                color = (0, 100, 255))
    
    # text to display the score of player 2
    cv2.putText(img = render, 
                text = "score : " + str(score_p2), 
                org = (int(1200/2 - 20),int(300/2)), 
                fontFace = cv2.FONT_HERSHEY_SIMPLEX, 
                fontScale = 1, 
                color = (189, 255, 0))    
    
    cv2.imshow(window_name, render) 
    #cv2.imshow('frame',render)
    
    # setting our terminating commands
    
    if (cv2.waitKey(1)==ord('q')):
        cv2.destroyAllWindows()
        break


    if(i==1000):
        cv2.destroyAllWindows()
        break
    
    



Its a draw
p2: Is decleared Winner
p2: Is decleared Winner
p1: Is decleared Winner
p2: Is decleared Winner
Its a draw
Its a draw
p1: Is decleared Winner
Its a draw
p2: Is decleared Winner
p2: Is decleared Winner
p1: Is decleared Winner
Its a draw
p1: Is decleared Winner
Its a draw
Its a draw
Its a draw
Its a draw
p1: Is decleared Winner
p2: Is decleared Winner
p2: Is decleared Winner
p1: Is decleared Winner
Its a draw
p1: Is decleared Winner
p1: Is decleared Winner
p2: Is decleared Winner
p1: Is decleared Winner
p1: Is decleared Winner
p1: Is decleared Winner
p2: Is decleared Winner
p1: Is decleared Winner
Its a draw
Its a draw
p1: Is decleared Winner
Its a draw
Its a draw
p1: Is decleared Winner
p1: Is decleared Winner
p2: Is decleared Winner
p1: Is decleared Winner
p2: Is decleared Winner
p2: Is decleared Winner
p1: Is decleared Winner
p1: Is decleared Winner
Its a draw
Its a draw
Its a draw
Its a draw
Its a draw
Its a draw
p1: Is decleared Winner
p1: Is decleared Winner
Its a draw


In [33]:
score_p1,score_p2

(4, 4)